In [ ]:
import pandas as pd
import logging
import psycopg2
from sqlalchemy import create_engine
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s:%(funcName)s:%(levelname)s:%(message)s')

# Define the PostgreSQL connection parameters
db_params = {
    'dbname': "InvoiceDB",
    'user': "postgres",
    'password': 12345,
    'host': 'localhost',  # Replace with your host
    'port': '5432',  # Replace with your port
}

# Establish a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')
table_names = ["DimClient", "DimDate", "DimMeter", "DimPayment"]

def get_data_from_postgresql(engine, table_names):
    dataframes = {}
    for table_name in table_names:
        query = f'SELECT * FROM "{table_name}"'
        dataframes[table_name] = pd.read_sql(query, engine)
    return dataframes

data = get_data_from_postgresql(engine, table_names)

DimClient = data["DimClient"]
DimDate = data["DimDate"]
DimMeter = data["DimMeter"]
DimPayment = data["DimPayment"]

# display(DimPayment)
# display(DimDate)

def create_fact_table(DimMeter, DimClient, DimPayment, DimDate):
    FactInvoice = pd.merge(DimMeter, DimClient, on="MeterID", how= "inner")
    FactInvoice['ClientID'] = FactInvoice['ClientID'].astype(str).str.zfill(4)
    FactInvoice['DateID'] = FactInvoice['Date'].dt.strftime('%Y%m').astype(str)
    FactInvoice['InvoiceID'] = FactInvoice['ClientID'] + FactInvoice['DateID']
    FactInvoice['From'] = FactInvoice['Date'] - pd.offsets.MonthBegin(1)
    FactInvoice['Till'] = FactInvoice['Date'] + pd.offsets.MonthEnd(0)
    FactInvoice['OutgoingBill'] = FactInvoice['Till'] + pd.offsets.MonthBegin(1)
    FactInvoice['PaymentDeadline'] = FactInvoice['OutgoingBill'] + pd.Timedelta(days=60)
    FactInvoice['KwhUsage'] = FactInvoice['Last_Meter_Reading'] - FactInvoice['Initial_Meter_Reading']
    # FactInvoice = pd.merge(DimDate, FactInvoice,  on="Date", how= "inner")
    # DimPayment['Date'] = pd.to_datetime(DimPayment['Date'])
    # FactInvoice = pd.merge(FactInvoice, DimPayment,  on=["Date","ClientID"], how='outer')
    display(FactInvoice)
    logging.info(' FactInvoice data successfully created')
    return FactInvoice

def load(FactInvoice):
    FactInvoice.to_sql('FactInvoice', engine, if_exists='replace', index=False)

if __name__ == '__main__':
    FactInvoice = create_fact_table(DimMeter, DimClient, DimPayment, DimDate)
    load(FactInvoice)
